In [10]:
import torch
from transformers import GPT2Tokenizer
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
model = AutoModelForCausalLMWithValueHead.from_pretrained("gpt2", device_map=device)
model_ref = AutoModelForCausalLMWithValueHead.from_pretrained("gpt2", device_map=device)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", device_map=device)
tokenizer.pad_token = tokenizer.eos_token

In [13]:
ppo_config = {"mini_batch_size": 1, "batch_size": 1}
config = PPOConfig(**ppo_config)
ppo_trainer = PPOTrainer(config, model, model_ref, tokenizer)

c:\Users\drobi\Documents\GitHub\all-in-training\.venv\Lib\site-packages\trl\trainer\ppo_trainer.py:266: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [14]:
query_txt = "This morning I went to the "
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(device)

In [15]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 20,
}
response_tensor = ppo_trainer.generate([item for item in query_tensor], return_prompt=False, **generation_kwargs)
response_txt = tokenizer.decode(response_tensor[0])

c:\Users\drobi\Documents\GitHub\all-in-training\.venv\Lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:650: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [16]:
reward = [torch.tensor(1.0, device=device)]

In [17]:
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

c:\Users\drobi\Documents\GitHub\all-in-training\.venv\Lib\site-packages\trl\trainer\ppo_trainer.py:1300: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ..\aten\src\ATen\native\ReduceOps.cpp:1807.)
  std_scores = data["scores"].std()
c:\Users\drobi\Documents\GitHub\all-in-training\.venv\Lib\site-packages\trl\trainer\ppo_trainer.py:1327: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ..\aten\src\ATen\native\ReduceOps.cpp:1807.)
  stats["tokens/queries_len_std"] = torch.std(query_lens).cpu().numpy().item()
c:\Users\drobi\Documents\GitHub\all-in-training\.venv\Lib\site-packages\trl\trainer\ppo_trainer.py:1330: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by o

In [20]:
decoded = tokenizer.batch_decode(response_tensor)
decoded

['vernacular and found myself at a bar, cook, with a wife. Buggas together in']